# <b> Recommender Systems </b>

## <b>Learning Objectives</b>

In this lesson, we will cover the following concepts:

- Recommendation system
- The long tail
- A simple popularity-based recommender system
- A collaborative filtering model
- Evaluating a recommendation system

### **What Is a Recommender System?**
 
A recommender or recommendation system is a subclass of an information filtering system that seeks to predict the rating or preference that a user would give to an item.

Let’s consider the example shown in the figure below. Here, we have a user database, that is, data consisting of items rated by the user. Now, let’s suppose that a new user visits and likes five out of ten items on the website. A recommender system recommends the items that the new user might like, based on similarity with other items. We will dive deeper  into this concept in the coming sections. 


![recommender_system_info](https://labcontent.simplicdn.net/data-content/content-assets/Data_and_AI/Applied_Machine_Learning/Images/Lesson_08_Recomendar_Systems/recommender_system_info.jpg)

## **The Theory of Long Tail**

- It shows how products in low demand or with low sales volume can collectively make up a market share that exceeds the relatively few current bestsellers and blockbusters but only if the store or distribution channel is large enough.

- The long tail concept looks at less popular goods in lower demand. The use of these goods could increase profitability as consumers  navigate  away from mainstream markets.

- This can be easily understood by looking at the figure [below](https://www.wired.com/2004/10/tail/).

![longtail](https://labcontent.simplicdn.net/data-content/content-assets/Data_and_AI/Applied_Machine_Learning/Images/Lesson_08_Recomendar_Systems/longtail.png)

The figure above clearly shows the use of long tail by [Rhapsody](https://en.wikipedia.org/wiki/Rhapsody_(music) where they sell music albums both online and off-line. We can clearly observe the following:
 
- Both Rhapsody and Walmart sell the most popular music albums online, but the former offers 19 times more songs than Walmart. Even though there is a demand for popular music albums, there is also a demand for the less popular online. Recommender systems leverage these less popular items online. 


## Recommend the Most Popular Items
 
- Let's consider the movie dataset. We will look carefully at the user ratings and think about what can be done.

- The answer that strikes first is the **most popular item**. This is exactly what we will be doing.

- Technically, this is the fastest method, but it does come with a major drawback, which is a lack of personalization. The dataset has many files; we will be looking at a few of them, mainly the ones that relate to movie ratings.

## **Popularity-Based Recommender System**

- There is a division by section, so the user can look at the section of his or her interest.

- At a time, there are only a few hot topics; there is a high chance that a user wants to read the news which is being read by most others.



### Import Libraries

In python, Pandas is used for data manipulation and analysis. NumPy is a package that includes a multidimensional array object and multiple derived objects. Matplotlib is an amazing visualization library in Python for 2D plots of arrays. Seaborn is an open-source Python library built on top of matplotlib. Mean_squared_error is a library that measures the average of the squares of the errors, which is the average squared difference between the estimated values and the actual value.

These libraries are written with the import keyword.



In [1]:
import pandas as pd
import os, io
import numpy as np
from pandas import Series, DataFrame, read_table
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error
%matplotlib inline


### Exporting Dataset from Zip File


Before reading data you need to download "ml-100k.zip" dataset from the resource section and upload it into the Lab. We will use Up arrow icon which is shown in the left side under View icon. Click on the Up arrow icon and upload the file wherever it is downloaded into your system.

After this you will see the downloaded file will be visible on the left side of your lab with all the .ipynb files.

Then, the below snippet will extract the zip dataset to the corresponding folder.



In [4]:
import zipfile
with zipfile.ZipFile('ml-100k.zip', 'r') as zip_ref:
    zip_ref.extractall(".")

We start to explore the data set of movie ratings and our interest lies particularly  in ratings. Let's see how we recommend the most popular (that is, highly rated) movies.

In [5]:
#Load the Ratings data
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = read_table('ml-100k//u.data',header=None,sep='\t')
ratings.columns = r_cols

i_cols = ['movie_id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = read_table('./ml-100k//u.item', sep='|',names=i_cols,
 encoding='latin-1')

In [6]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


In [9]:
items.head()

,movie_id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0



Ratings is a variable that stores all the columns from the ml-100k dataset in the u.data file.
 The head() function displays the first five rows from ratings.



## Let's Build a Popularity-Based Recommender System

With our initial exploration, we decided that ideal data would be the one where we could also have the movie ratings with us. Let's see how we are able to do this.



We will use the pd.merge function that is used to combine data on common columns or indices.



In [7]:
new_data = pd.merge(items,ratings,on='movie_id')
new_data  = new_data[['movie_id','movie title','user_id','rating']]

In [8]:
new_data.head()

,movie_id,movie title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3



New_data is a variable that stores data read by the pd.merge function. It consists of items and ratings.
 The head() function displays the first five rows from new_data.



Before proceeding to build the recommender system, we will observe the following steps to recommend movies:
- Find unique users
- Count the number of times the movie has been seen
- [Rank](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rank.html) the scores (counts)

In [10]:
def popularity(train,title,ids):
    train_data_grouped = train.groupby([title])[ids].count().reset_index()  #user_id  #movie title
    train_data_grouped.rename(columns = {ids: 'score'},inplace=True)            
    train_data_sort = train_data_grouped.sort_values(['score',title], ascending = [0,1])
    train_data_sort['Rank'] = train_data_sort['score'].rank(ascending=0, method='first')
    popularity_recommendations = train_data_sort.head(10) 
    return popularity_recommendations

In [11]:
popularity(new_data,'movie title','user_id')

,movie title,score,Rank
1398,Star Wars (1977),584,1.0
333,Contact (1997),509,2.0
498,Fargo (1996),508,3.0
1234,Return of the Jedi (1983),507,4.0
860,Liar Liar (1997),485,5.0
460,"English Patient, The (1996)",481,6.0
1284,Scream (1996),478,7.0
1523,Toy Story (1995),452,8.0
32,Air Force One (1997),431,9.0
744,Independence Day (ID4) (1996),429,10.0


![Simplilearn_Logo](https://labcontent.simplicdn.net/data-content/content-assets/Data_and_AI/Logo_Powered_By_Simplilearn/SL_Logo_1.png)